## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is essentially the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

- Apply the map(func) transformation to a given function on all elements of an RDD in different partitions 
- Apply a map transformation for all elements of an RDD 
- Compare the difference between a transformation and an action within RDDs 
- Use collect(), count(), and take() actions to trigger spark transformations  
- Use filter to select data that meets certain specifications within an RDD 
- Set number of partitions for parallelizing RDDs 
- Create RDDs from Python collections 


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies while performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified. 
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new dataset from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [1]:
data = range(1,1001)
len(data)

# 1000

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use `'local[*]'` as the master.

In [2]:
import pyspark

sc = pyspark.SparkContext('local[*]')

Once you've created the SparkContext, you can use the `.parallelize()` method to create an RDD that will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [4]:
rdd = sc.parallelize(data, numSlices=10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.PipelinedRDD'>


Determine how many partitions are being used with this RDD with the `.getNumPartitions()` method.

In [5]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first `n` items in the RDD
* `top`: returns the top `n` items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [6]:
# count
rdd.count()

1000

In [7]:
# first
rdd.first()

1

In [11]:
# take
rdd.take(5)

[1, 2, 3, 4, 5]

In [12]:
# top
rdd.top(5)

[1000, 999, 998, 997, 996]

In [13]:
# collect
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 with a random number from 0-1.

In [14]:
import random
import numpy as np

nums = np.array(range(1, 1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([3.58563855e-01, 1.93970670e+00, 2.90161001e+00, 3.02140896e+00,
       2.28384952e+00, 2.35009324e+00, 5.38311818e+00, 2.16372558e+00,
       2.81478628e+00, 3.13205135e-01, 2.54049740e-01, 2.46835938e+00,
       5.65948159e-01, 3.09060541e+00, 6.00424468e+00, 5.66217844e+00,
       1.55553832e+01, 5.23903781e+00, 1.82849576e+01, 1.08131744e+01,
       1.88028390e+01, 7.10291553e+00, 1.85790216e+01, 9.37154525e+00,
       1.63226469e+01, 1.13515138e+00, 8.61175071e+00, 3.84610910e+00,
       2.77665215e+01, 9.86028866e+00, 1.40703802e+00, 1.86105331e+00,
       2.80539832e+01, 6.55076642e+00, 1.69887055e+01, 1.14125664e+01,
       3.68958922e+01, 3.57905038e+01, 8.56985642e+00, 9.61088393e+00,
       1.32670309e+01, 3.17727653e+01, 2.40724987e+01, 3.72616646e+01,
       3.61122078e+01, 1.98899361e+01, 1.93803975e+01, 8.91347691e+00,
       1.43622072e+01, 2.29688244e+00, 3.13566047e+00, 5.18874157e+01,
       1.43737605e+01, 2.87791387e+01, 4.90254100e+01, 3.79224377e+01,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [16]:
price_items = sc.parallelize(sales_figures, numSlices=10)


Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax()` that returns the amount of money our company will receive after the sales tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the `.map()` method and assign it to a variable `renenue_minus_tax`

In [17]:
def sales_tax(num):
    return num*0.92

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax()` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [18]:
# perform action to retrieve rdd values
revenue_minus_tax.take(10)

[0.3298787462450907,
 1.784530159817816,
 2.6694812072777245,
 2.7796962442831816,
 2.1011415610392072,
 2.1620857833918685,
 4.952468726212794,
 1.9906275292952833,
 2.589603380802789,
 0.28814872415343906]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a `.map()` method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [19]:
discounted = revenue_minus_tax.map(lambda x: x*.9)

In [20]:
discounted.take(10)

[0.29689087162058164,
 1.6060771438360344,
 2.402533086549952,
 2.5017266198548636,
 1.8910274049352866,
 1.9458772050526818,
 4.457221853591515,
 1.791564776365755,
 2.33064304272251,
 0.25933385173809514]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff and use a collection method to see the 15 costliest items.

In [21]:
price_items.map(sales_tax).map(lambda x: x*0.9).top(15)

[805.4484591335402,
 800.4965417373337,
 796.236781400869,
 785.2561311504691,
 773.3464824853495,
 753.8513819242646,
 748.0883156393166,
 743.0403604854903,
 733.281477738186,
 731.1776736699211,
 726.5131526524899,
 717.5599133520916,
 714.633470637429,
 714.4867462066469,
 705.2381292575303]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [22]:
discounted.toDebugString()

b'(10) PythonRDD[10] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[6] at readRDDFromFile at PythonRDD.scala:262 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use `.flatMap()` rather than a simple `.map()`. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will return a tuple with (original price, post-discount price).

In [23]:
mapped =price_items.map(lambda x: (x, x*0.92*0.9))
print(mapped.count())
print(mapped.take(10))

1000
[(0.358563854614229, 0.29689087162058164), (1.9397066954541478, 1.6060771438360344), (2.9016100079105698, 2.402533086549952), (3.021408961177371, 2.5017266198548636), (2.2838495228687035, 1.8910274049352866), (2.3500932428172483, 1.9458772050526818), (5.383118180666081, 4.457221853591515), (2.16372557532096, 1.791564776365755), (2.8147862834812925, 2.33064304272251), (0.3132051349493903, 0.25933385173809514)]


Note that we have 1000 tuples created to our specification. Let's take a look at how `.flatMap()` differs in its implementation. Use the `.flatMap()` method with the same function you created above.

In [24]:
flat_mapped = price_items.flatMap(lambda x: (x, x*0.92*0.9))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.358563854614229, 0.29689087162058164, 1.9397066954541478, 1.6060771438360344, 2.9016100079105698, 2.402533086549952, 3.021408961177371, 2.5017266198548636, 2.2838495228687035, 1.8910274049352866]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use `.flatMap()` rather than `.map()` in order to properly reduce to our specifications. This is not one of those instances, but in the upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the `.filter()` method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a `.filter()` method to meet the consultant's suggestion's specifications. set `RDD = selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [25]:
# use the filter function
selected_items = discounted.filter(lambda x: x>300)
selected_items.count()

# calculate total remaining in inventory 


267

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use the `.reduce()` method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [26]:
selected_items.reduce(lambda x,y: x+y)

124245.7586732042

In [30]:
np.sum(selected_items.collect())

124245.75867320423

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [31]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1, 50), x))

sales_data.take(7)

[(47, 304.6221167134614),
 (16, 302.2534980055785),
 (21, 309.4188871261852),
 (3, 300.75310849085224),
 (22, 311.014577326402),
 (20, 322.9717367302373),
 (46, 332.590353119003)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called `.reduceByKey()` to perform reducing operations while grouping by keys. After you have calculated the total, use the `.sortBy()` method on the RDD to rank the users from the highest spending to the least spending. 

In [32]:
# calculate how much each user spent
total_spent = sales_data.reduceByKey(lambda x, y: x+y)
total_spent.take(10)

[(20, 3548.8171257740632),
 (40, 2995.0412213755435),
 (10, 5681.973924337568),
 (50, 3441.7637937225886),
 (30, 1735.2562895373858),
 (41, 1070.147321370843),
 (1, 2262.2683189802465),
 (31, 1471.177988380965),
 (21, 2740.553050014655),
 (11, 3282.3701794779677)]

In [35]:
# sort the users from highest to lowest spenders
total_spent.sortBy(lambda x: x[1], ascending=False).collect()

[(10, 5681.973924337568),
 (4, 5561.815825639957),
 (9, 4290.932877288963),
 (49, 4226.388195550973),
 (47, 4092.9609454604406),
 (14, 3972.691438531422),
 (34, 3723.557545918017),
 (17, 3569.805945587463),
 (20, 3548.8171257740632),
 (18, 3510.8834740165275),
 (50, 3441.7637937225886),
 (11, 3282.3701794779677),
 (39, 3275.5487932227115),
 (8, 3192.2433446963787),
 (40, 2995.0412213755435),
 (44, 2890.9145395121996),
 (46, 2864.1297660580512),
 (32, 2778.8589213219147),
 (21, 2740.553050014655),
 (25, 2719.0456741163844),
 (45, 2668.62604867923),
 (15, 2651.988596338242),
 (28, 2432.653318089462),
 (7, 2302.9127896165573),
 (1, 2262.2683189802465),
 (24, 2209.157738982251),
 (38, 2197.6517756631893),
 (22, 2196.5702029664076),
 (36, 2191.2563169460127),
 (2, 2061.5764313256695),
 (35, 2026.4966960938325),
 (29, 1955.9297715302218),
 (23, 1911.5077854614538),
 (27, 1806.7335206596701),
 (3, 1748.8418990231426),
 (12, 1748.6320805664632),
 (26, 1739.5264157104173),
 (30, 1735.2562895373

Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [39]:
total_items = sales_data.countByKey()
sorted(total_items.items(), key=lambda x:x[1], reverse=True)

[(45, 11),
 (44, 10),
 (12, 10),
 (31, 9),
 (42, 9),
 (36, 8),
 (27, 8),
 (14, 8),
 (37, 8),
 (39, 8),
 (28, 7),
 (1, 7),
 (19, 7),
 (43, 7),
 (4, 7),
 (30, 7),
 (13, 7),
 (18, 6),
 (17, 6),
 (41, 6),
 (22, 6),
 (8, 6),
 (5, 6),
 (47, 5),
 (34, 5),
 (10, 5),
 (35, 5),
 (23, 5),
 (48, 5),
 (49, 5),
 (7, 5),
 (6, 4),
 (33, 4),
 (24, 4),
 (29, 3),
 (9, 3),
 (21, 3),
 (46, 3),
 (25, 3),
 (32, 3),
 (11, 3),
 (3, 3),
 (50, 3),
 (16, 2),
 (2, 2),
 (26, 2),
 (20, 2),
 (38, 2),
 (40, 2),
 (15, 2)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD and extracting lineage. We also used transformations and actions to perform calculations across RDDs on a distributed setup. In the next lab, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
